In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from scipy.spatial.distance import euclidean
dt = pd.read_csv('ucicancer.csv')

In [2]:
dt.head()

,Unnamed: 0,a,b,c,d,e,f,g,h,i,j,class
0,0,1002945,5,4,4,5,7,10,3,2,1,2
1,1,1015425,3,1,1,1,2,2,3,1,1,2
2,2,1016277,6,8,8,1,3,4,3,7,1,2
3,3,1017023,4,1,1,3,2,1,3,1,1,2
4,4,1017122,8,10,10,8,7,10,9,7,1,4


In [3]:
dt.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
dt.head()

,a,b,c,d,e,f,g,h,i,j,class
0,1002945,5,4,4,5,7,10,3,2,1,2
1,1015425,3,1,1,1,2,2,3,1,1,2
2,1016277,6,8,8,1,3,4,3,7,1,2
3,1017023,4,1,1,3,2,1,3,1,1,2
4,1017122,8,10,10,8,7,10,9,7,1,4


In [5]:
dt.dropna().shape,dt.shape

((698, 11), (698, 11))

In [6]:
dt.dtypes

a         int64
b         int64
c         int64
d         int64
e         int64
f         int64
g        object
h         int64
i         int64
j         int64
class     int64
dtype: object

In [7]:
dt['g'].describe()

count     698
unique     11
top         1
freq      401
Name: g, dtype: object

In [8]:
dt['g'].unique()

array(['10', '2', '4', '1', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

In [9]:
dt=dt.replace('?',15)

In [10]:
dt['g'] = dt['g'].astype(dtype='i')

In [11]:
dt.dtypes

a        int64
b        int64
c        int64
d        int64
e        int64
f        int64
g        int32
h        int64
i        int64
j        int64
class    int64
dtype: object

In [12]:
x = dt.drop(['a','class'],axis=1)
y = dt['class']

In [13]:
from sklearn.model_selection import train_test_split
x_tr,x_ts,y_tr,y_ts = train_test_split(x,y,test_size=0.2)
x_tr.shape,y_tr.shape,x_ts.shape,y_ts.shape

((558, 9), (558,), (140, 9), (140,))

In [14]:
d=dict.fromkeys(y_tr)
d

{4: None, 2: None}

In [15]:
for i in d:
    d[i]=[]
d

{4: [], 2: []}

In [16]:
for i in d:
    d[i]=[]
for i in range(len(x_tr)):
    d[y_tr.iloc[i]].append(x_tr.iloc[i].values)
d

{4: [array([ 6, 10, 10, 10,  8, 10, 10, 10,  7], dtype=int64),
  array([ 5,  5,  5,  8, 10,  8,  7,  3,  7], dtype=int64),
  array([ 7,  6,  3,  2,  5, 10,  7,  4,  6], dtype=int64),
  array([ 3, 10,  8,  7,  6,  9,  9,  3,  8], dtype=int64),
  array([10,  4,  4,  6,  2, 10,  2,  3,  1], dtype=int64),
  array([ 5, 10, 10, 10,  6, 10,  6,  5,  2], dtype=int64),
  array([10,  7,  7,  4,  5, 10,  5,  7,  2], dtype=int64),
  array([ 6, 10, 10,  2,  8, 10,  7,  3,  3], dtype=int64),
  array([ 8, 10, 10, 10,  5, 10,  8, 10,  6], dtype=int64),
  array([ 5,  4,  4,  9,  2, 10,  5,  6,  1], dtype=int64),
  array([ 8, 10,  5,  3,  8,  4,  4, 10,  3], dtype=int64),
  array([8, 3, 8, 3, 4, 9, 8, 9, 8], dtype=int64),
  array([10,  5,  8, 10,  3, 10,  5,  1,  3], dtype=int64),
  array([ 8,  8,  9,  4,  5, 10,  7,  8,  1], dtype=int64),
  array([10, 10, 10,  8,  2, 10,  4,  1,  1], dtype=int64),
  array([5, 3, 3, 3, 2, 3, 4, 4, 1], dtype=int64),
  array([ 9, 10, 10, 10, 10,  5, 10, 10, 10], dtype=int

In [17]:
def knn2(dt,prd):
    dist=[]
    for grp in dt:
        for pt in dt[grp]:
            dist.append((euclidean(pt,prd),grp))
    votes=[i[1] for i in sorted(dist)[:3]]
    return Counter(votes).most_common(1)[0][0]

In [19]:
#score
c=0
for i in range(len(x_ts)):
    if knn2(d,x_ts.iloc[i])==y_ts.iloc[i]:
        c+=1
c/len(x_ts)

0.9571428571428572

In [20]:
import warnings
class KNN:
    def __init__(s,k=5):
        s.k=k
    def fit(s,x_tr,y_tr):
        s.d=dict.fromkeys(y_tr)
        for i in s.d:
            s.d[i]=[]
        for i in range(len(x_tr)):
            s.d[y_tr.iloc[i]].append(x_tr.iloc[i].values)
    def predict(s,prd):
        if len(prd.shape)!=2:
            warnings.warn("use 2d array else .reshape(1,-1)")
        else:
            output=[]
            for i in prd:
                dist=[]
                for grp in s.d:
                    for pt in s.d[grp]:
                        dist.append((euclidean(pt,prd),grp))
                votes=[i[1] for i in sorted(dist)[:s.k]]
                output.append(Counter(votes).most_common(1)[0][0])
            return output
    def score(s,x_ts,y_ts):
        c=0
        for i in range(len(x_ts)):
            if s.predict(x_ts.iloc[i].values.reshape(1,-1))==y_ts.iloc[i]:
                c+=1
        return c/len(x_ts)

In [21]:
algo2=KNN()
algo2.fit(x_tr,y_tr)
algo2.score(x_ts,y_ts)

0.9642857142857143